## Huber loss function

In [31]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import backend as K

In [18]:
def huber_loss(y_true, y_pred):
  threshold = 1
  error = y_true - y_pred
  is_small_error = tf.abs(error) <= threshold
  small_error_loss = tf.square(error) / 2
  big_error_loss = threshold * (tf.abs(error) - threshold/2.)
  return tf.where(is_small_error, tf.cast(small_error_loss, dtype=tf.float32), tf.cast(big_error_loss, dtype=tf.float32))

In [19]:
X = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
y = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[1])
])
model.compile(loss="mse",
              optimizer="sgd")
model.fit(X, y, epochs=500, verbose=0)
print(model.predict([10.0]))

1/1 [==============================] - 0s 106ms/step
[[18.97529]]


In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[1])
])

model.compile(loss=huber_loss,
              optimizer="sgd")
model.fit(X, y, epochs=500, verbose=0)
print(model.predict([10.0]))

1/1 [==============================] - 0s 68ms/step
[[18.754059]]


In [23]:
def huber_loss_with_threshold(threshold):
  def huber_loss(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error)/2
    big_error_loss = threshold * (tf.abs(error) - threshold/2)
    return tf.where(is_small_error, small_error_loss, big_error_loss)
  return huber_loss


In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[1])
])
model.compile(loss=huber_loss_with_threshold(1.5),
              optimizer="sgd")
model.fit(X, y,
          epochs=500,
          verbose=0)

model.predict([10.0])

1/1 [==============================] - 0s 74ms/step


array([[18.540209]], dtype=float32)

In [27]:
class MyHuberLoss(tf.keras.losses.Loss):
  def __init__(self, threshold=1):
    super().__init__()
    self.threshold=threshold

  def call(self, y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= self.threshold
    small_error_loss = tf.square(error)/2
    big_error_loss = self.threshold * (tf.abs(error) - self.threshold/2)
    return tf.where(is_small_error, small_error_loss, big_error_loss)


In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[1])
])
model.compile(loss=MyHuberLoss(threshold=2),
              optimizer="sgd")
model.fit(X, y,
         epochs=500,
         verbose=0)
model.predict([10.0])

1/1 [==============================] - 0s 99ms/step


array([[18.786963]], dtype=float32)

## Contrastive loss

In [32]:

def contrastive_loss(y_true, y_pred):
  margin = 1
  square_pred = K.square(y_pred)
  margin_square = K.square(K.maximum(margin - y_pred, 0))
  return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [34]:
def contrastive_loss_with_margin(margin=1):
  def contrastive_loss(y_true, y_pred):
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin-y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)
  return contrastive_loss

In [36]:
class ContrastiveLoss(tf.keras.losses.Loss):
  def __init__(self, margin):
    super().__init__()
    self.margin = margin

  def call(self, y_true, y_pred):
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(self.margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

## Define the custom loss function (TODO)

my_rmse

In [39]:
!wget https://raw.githubusercontent.com/pabaq/Coursera-TensorFlow-Advanced-Techniques-Specialization/main/C1-Custom-Models-Layers-and-Loss-Functions-with-TensorFlow/W2-Custom-Loss-Functions/utils.py

--2024-03-30 16:04:11--  https://raw.githubusercontent.com/pabaq/Coursera-TensorFlow-Advanced-Techniques-Specialization/main/C1-Custom-Models-Layers-and-Loss-Functions-with-TensorFlow/W2-Custom-Loss-Functions/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2248 (2.2K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   2.20K  --.-KB/s    in 0s      

2024-03-30 16:04:12 (25.0 MB/s) - ‘utils.py’ saved [2248/2248]



In [40]:
import utils

In [38]:
def my_rmse(y_true, y_pred):
  error = y_true - y_pred
  sqr_error = K.square(error)
  mean_sqr_error = K.mean(sqr_error)
  return K.sqrt(mean_sqr_error)

In [41]:
utils.test_my_rmse(my_rmse)

 All public tests passed


In [42]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[1])
])

model.compile(loss=my_rmse,
              optimizer='sgd')
model.fit(X,
          y,
          epochs=500,
          verbose=0)
model.predict([10.0])

1/1 [==============================] - 0s 65ms/step


array([[18.88435]], dtype=float32)

In [43]:
utils.test_model_loss(model.loss)

 All public tests passed
